In [1]:
import matplotlib.pyplot as plt
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

num_classes = 10
im_rows = 32
im_cols = 32
in_shape = (im_rows, im_cols, 3)

# データを読み込み
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# データを正規化
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
# ラベルデータをOne-hot形式に変換
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# モデルを定義
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=in_shape))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 学習を実行
hist = model.fit(X_train, y_train, batch_size=32, epochs=50, verbose=1, validation_data=(X_test, y_test))

# モデルを評価
score = model.evaluate(X_test, y_test, verbose=1)
print('正解率=', score[1], 'loss=', score[0])

Using TensorFlow backend.


Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 109s 2ms/step - loss: 1.5240 - accuracy: 0.4420 - val_loss: 1.1734 - val_accuracy: 0.5775
Epoch 2/50
50000/50000 [==============================] - 107s 2ms/step - loss: 1.1158 - accuracy: 0.6056 - val_loss: 0.9483 - val_accuracy: 0.6721
Epoch 3/50
50000/50000 [==============================] - 106s 2ms/step - loss: 0.9679 - accuracy: 0.6584 - val_loss: 0.8582 - val_accuracy: 0.6994
Epoch 4/50
50000/50000 [==============================] - 107s 2ms/step - loss: 0.8877 - accuracy: 0.6880 - val_loss: 0.8098 - val_accuracy: 0.7179
Epoch 5/50
50000/50000 [==============================] - 107s 2ms/step - loss: 0.8249 - accuracy: 0.7123 - val_loss: 0.7516 - val_accuracy: 0.7398
Epoch 6/50
50000/50000 [==============================] - 107s 2ms/step - loss: 0.7768 - accuracy: 0.7290 - val_loss: 0.7277 - val_accuracy: 0.7517
Epoch 7/50
50000/50000 [==============================] - 108s

### 学習結果を保存

In [2]:
model.save_weights('cifar10-weight.h5')

In [ ]:
import cv2
import numpy as np

labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
im_size = 32 * 32 * 3

# モデルデータを読み込み
model.load_weights('cifar10-weight.h5')

# OpenCvを使って、画像を読み込み
im = cv2.imread('car-test.jpg')
# 色空間を変換して、リサイズ
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
im = cv2.resize(im, (32, 32))
plt.imshow(im) # 画像を出力
plt.show()

# MLPで学習した画像データに合わせる
im = im.reshape(im_size).astype('float32') / 255
# 予測する
r = model.predict(np.array([im]), batch_size=32, verbose=1)
res = r[0]
# 結果を表示する
for i, acc in enumerate(res):
    print(labels[i], '=', int(acc * 100))
print('---')
print('予測した結果=', labels[res.argmax()])